-First we import some important packages below. Afterwards, we set up our request to get our website source as a text file, followed by it's soup object.

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [3]:
soup = BeautifulSoup(data,"html5lib")

Now, we'll find the table in our soup object and iterate throughout each cell of the table to retreive the postal code, borough, and neighborhood. We will save the results to an empty dictionary and append each dictionary we receive to an empty set (contents).

In [4]:
contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(cell)

In [5]:
print(contents)

[{'Postal Code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'Postal Code': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'Postal Code': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'Postal Code': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'Postal Code': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'Postal Code': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'Postal Code': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'Postal Code': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'Postal Code': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'Postal Code': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'Postal Code': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'Posta

In [6]:
#turn the results into a dataframe we can manipulate
df = pd.DataFrame(contents)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
print ("We have {} rows in our dataframe.".format(df.shape[0]))

We have 103 rows in our dataframe.


-- This is where part 2 begins --

I downloaded the geospatial coordinates to my computer, so i'll turn the results into a dataframe

In [8]:
r = pd.read_csv(r'C:\Users\adardick\Downloads\Geospatial_Coordinates.csv')

In [9]:
r

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Now, I'm gonna combine it with my boroughs and neighborhoods dataframe where the postal codes are the same, so we get coordinates for each borough

In [10]:
DF = pd.merge(df, r, on='Postal Code')
DF

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


-- This is where part 3 begins --

In [11]:
pd.set_option('display.max_rows', None)


In [12]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install folium==0.5.0

Note: you may need to restart the kernel to use updated packages.


In [14]:
import random
from geopy.geocoders import Nominatim
import folium

In my analysis, I'm going to cluster the neighborhoods of Toronto up into unique groups based on their most common types of venues. We'll start by defining the URL to my foursquare account, and then defining the function that will take the values of our dataframe (the neighborhoods of Toronto) and give us a dataframe of all the resulting venues within a 500 meter radius of each neighborhood.

In [15]:
client_ID = 'N1S1NLEIDD2O4YC10EZ2YONJX5NADGZDM2MDPDHDKWW2NZRH'
secret = 'QDUA3YVZJFXD0OZEV14PS42OW0MJ2BGNAELYKFXVUDAI4YHZ'
version = '20180604'
limit = 30

In [16]:
def getnearbyVenues(neighborhood, lats, lons, radius=500):
    venues = []
    for name, lat, lng in zip(neighborhood, lats, lons):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_ID,
            secret,
            version,
            lat,
            lng,
            radius,
            limit)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
    
        venues.append([(
            name,
            lat,
            lng,
            i['venue']['name'],
            i['venue']['location']['lat'],
            i['venue']['location']['lng'],
            i['venue']['categories'][0]['name']) for i in results])
        
    nearby_venues = pd.DataFrame([item for venue in venues for item in venue])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue Name',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']
    
     
    return (nearby_venues)

In [17]:
toronto_venues = getnearbyVenues(DF['Neighborhood'], DF['Latitude'], DF['Longitude'])

In [18]:
print (toronto_venues.shape)

(1330, 7)


In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",24,24,24,24,24,24
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15


In [45]:
print('There are {} unique categories of venue.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 unique categories of venue.


In [46]:
toronto_venues.shape

(1330, 7)

Now that we have all of the venues, we're gonna use one-hot encoding to quantize each category so we can break down each neighborhood into how often each venue comes up.

In [47]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
                                               
columnss = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
                                               
toronto_onehot = toronto_onehot[columnss]

In [48]:
toronto_groups = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_groups

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
1,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
2,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
4,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
5,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.033333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
6,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
7,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
8,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00
9,Caledonia-Fairbanks,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25


This is where we define the function that sorts all venues from most common to least common. Afterwards, we're gonna create a dataframe that lists the most common venues of each neighborhood. 

In [49]:
def most_common_venues(row, num_top):
    row_cat = row.iloc[1:]
    row_cat_sort = row_cat.sort_values(ascending=False)
    
    return row_cat_sort.index.values[0:num_top]

In [50]:
num_top = 5
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for i in np.arange(num_top):
    try:
        columns.append('{}{} most common venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th most common venue'.format(i+1))
        
sorted_toronto = pd.DataFrame(columns=columns)
sorted_toronto['Neighborhood'] = toronto_groups['Neighborhood']

In [51]:
for k in np.arange(toronto_groups.shape[0]):
    sorted_toronto.iloc[k, 1:] = most_common_venues(toronto_groups.iloc[k, :], num_top)

    
sorted_toronto.head()

,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Department Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Sandwich Place,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Mobile Phone Shop,Supermarket
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Pharmacy,Thai Restaurant


Now that we have a dataframe of the most common venues of each neighborhood, we want to be able to segment these neighborhoods based on the kinds of venues they have around them. We'll use k-clustering to cluster the neighborhoods then visualize it on a map.

In [52]:
from sklearn.cluster import KMeans
kclusters = 4

toronto_clusters = toronto_groups.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state= 0).fit(toronto_clusters)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2])

I had messed around with sorted_toronto a bit and already created the 'Cluster Label' column, so that bit of the code is not present. (original code was sorted_toronto.insert(0, 'Cluster Label', kmeans.labels_)) Afterwards we combine the cluster labels with the sorted popular venues.

In [53]:
sorted_toronto['Cluster Label']= kmeans.labels_

complete_toronto = DF

complete_toronto = complete_toronto.join(sorted_toronto.set_index('Neighborhood'), on='Neighborhood')

complete_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,Cluster Label
0,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Deli / Bodega,2.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Intersection,1.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,1.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Clothing Store,Coffee Shop,Furniture / Home Store,Accessories Store,Event Space,1.0
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,Coffee Shop,Diner,Café,Sushi Restaurant,Burrito Place,1.0
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Women's Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,1.0
7,M3B,North York,Don Mills North,43.745906,-79.352188,Athletics & Sports,Japanese Restaurant,Café,Caribbean Restaurant,Dessert Shop,1.0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Place,Pharmacy,Flea Market,Athletics & Sports,Intersection,1.0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Coffee Shop,Café,Clothing Store,Gastropub,Sandwich Place,1.0


Since some cells have no values, we'll just drop them. 

In [54]:
completed_toronto = complete_toronto.drop(labels=[5, 45, 50, 95], axis=0).reset_index()

In [55]:
completed_toronto

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,Cluster Label
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Deli / Bodega,2.0
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Intersection,1.0
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,1.0
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Clothing Store,Coffee Shop,Furniture / Home Store,Accessories Store,Event Space,1.0
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,Coffee Shop,Diner,Café,Sushi Restaurant,Burrito Place,1.0
5,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Women's Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,1.0
6,7,M3B,North York,Don Mills North,43.745906,-79.352188,Athletics & Sports,Japanese Restaurant,Café,Caribbean Restaurant,Dessert Shop,1.0
7,8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Place,Pharmacy,Flea Market,Athletics & Sports,Intersection,1.0
8,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Coffee Shop,Café,Clothing Store,Gastropub,Sandwich Place,1.0
9,10,M6B,North York,Glencairn,43.709577,-79.445073,Japanese Restaurant,Asian Restaurant,Park,Bakery,Women's Store,1.0


I found a random address in the middle of toronto to use as my starting address

In [56]:
address = '770 Don Mills Road North York, ON, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [57]:
import matplotlib.cm as cm
import matplotlib.colors as colors

When creating the map, i wanted to use distinguishable colors, so i chose red, blue, orange, and black. 

In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
rainbow = ['blue', 'red', 'orange', 'black']

marker_color = []
for lat, lon, poi, cluster in zip(completed_toronto['Latitude'], completed_toronto['Longitude'], completed_toronto['Neighborhood'], completed_toronto['Cluster Label']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color = rainbow[int(cluster-1)],
        fill_opacity = 0.7).add_to(map_clusters)

map_clusters

As we can see, after Toronto is split into 4 distinct clusters, we can see many neighborhoods fall into the cluster that's colored blue. If we group up the dataframe by it's cluster label, perhaps we can see some characteristics of each cluster.

In [59]:
one_clust = completed_toronto[completed_toronto['Cluster Label']==0]
two_clust = completed_toronto[completed_toronto['Cluster Label']==1]
three_clust = completed_toronto[completed_toronto['Cluster Label']==2]
four_clust = completed_toronto[completed_toronto['Cluster Label']==3]
one_clust

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,Cluster Label
21,22,M1G,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Korean BBQ Restaurant,Escape Room,Eastern European Restaurant,Dumpling Restaurant,0.0
23,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Coffee Shop,Spa,Café,Park,Bubble Tea Shop,0.0
37,38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,Coffee Shop,Chinese Restaurant,Discount Store,Hobby Shop,Department Store,0.0
48,51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,Motel,American Restaurant,Dance Studio,Dumpling Restaurant,Drugstore,0.0
53,56,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,Coffee Shop,Turkish Restaurant,Discount Store,Sandwich Place,Dance Studio,0.0
57,60,M3N,North York,Downsview Northwest,43.761631,-79.520999,Coffee Shop,Athletics & Sports,Grocery Store,Discount Store,Eastern European Restaurant,0.0
69,72,M2R,North York,Willowdale West,43.782736,-79.442259,Coffee Shop,Supermarket,Pharmacy,Butcher,Discount Store,0.0
73,76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W,43.636966,-79.615819,Coffee Shop,Hotel,American Restaurant,Middle Eastern Restaurant,Sandwich Place,0.0


In [60]:
two_clust

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,Cluster Label
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,Coffee Shop,Portuguese Restaurant,French Restaurant,Hockey Arena,Intersection,1.0
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,1.0
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Clothing Store,Coffee Shop,Furniture / Home Store,Accessories Store,Event Space,1.0
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,Coffee Shop,Diner,Café,Sushi Restaurant,Burrito Place,1.0
5,6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Women's Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,1.0
6,7,M3B,North York,Don Mills North,43.745906,-79.352188,Athletics & Sports,Japanese Restaurant,Café,Caribbean Restaurant,Dessert Shop,1.0
7,8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Pizza Place,Pharmacy,Flea Market,Athletics & Sports,Intersection,1.0
8,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Coffee Shop,Café,Clothing Store,Gastropub,Sandwich Place,1.0
9,10,M6B,North York,Glencairn,43.709577,-79.445073,Japanese Restaurant,Asian Restaurant,Park,Bakery,Women's Store,1.0
10,11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,Pool,Bakery,Middle Eastern Restaurant,Home Service,Women's Store,1.0


In [61]:
three_clust

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,Cluster Label
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,Park,Fast Food Restaurant,Food & Drink Shop,Women's Store,Deli / Bodega,2.0
20,21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,Park,Women's Store,Pool,Dance Studio,Dumpling Restaurant,2.0
34,35,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,Park,Convenience Store,Women's Store,Deli / Bodega,Dumpling Restaurant,2.0
49,52,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,Park,Women's Store,Dance Studio,Dumpling Restaurant,Drugstore,2.0
61,64,M9N,York,Weston,43.706876,-79.518188,Park,Women's Store,Dance Studio,Dumpling Restaurant,Drugstore,2.0
63,66,M2P,North York,York Mills West,43.752758,-79.400049,Park,Convenience Store,Women's Store,Deli / Bodega,Dumpling Restaurant,2.0
74,77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,Sandwich Place,Mobile Phone Shop,Park,Drugstore,Donut Shop,2.0
80,83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,Park,Restaurant,Tennis Court,Women's Store,Dance Studio,2.0
82,85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,Playground,Park,Intersection,Dance Studio,Drugstore,2.0
88,91,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,Park,Playground,Trail,Curling Ice,Drugstore,2.0


In [62]:
four_clust

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,Cluster Label
54,57,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,Baseball Field,Women's Store,Escape Room,Eastern European Restaurant,Dumpling Restaurant,3.0


Based on what I see in the clusters I would characterize each cluster as the following:

Cluser 1: This cluster has all the neighborhoods whose most popular venue is either a motel or coffee shop

Cluster 2: This cluster has an assortment of different venues and contains the neighborhoods where you would want anything else you could think of.

Cluster 3: These are the neighborhoods you would want to go to if you wanted to live near a park.

Cluster 4: This is the neighborhood you would want to go to if you wanted many baseball fields, women's stores, and escape rooms.